In [1]:
import pandas as pd
import MetaTrader5 as mt5
import talib as ta
from bb_setup import setup_2,setup_1
import warnings
from datetime import datetime,timedelta
warnings.filterwarnings("ignore")

In [11]:
pontos = 60
rate_stop = 1
rate_tp = 3
spread = 5
var_bb = 1.8

timeframe = 5
                
ativo = f'win_{timeframe}_full'
colnames = ['Data','Open','High','Low','Close','Vol']
df = pd.read_csv(f'./Dados/{ativo}.csv', names = colnames, skiprows=1)
df = df[(df['Data'] > '2022-12-01') & (df['Data'] < '2022-12-30')]

df_ticks = pd.read_csv('./Dados/tick_win_11_12.csv')

df_acao = setup_1(df,df_ticks,var_bb = var_bb,time_period = 20,pontos = pontos,rate_stop = 1,rate_tp = rate_tp,timeframe = timeframe)


In [12]:
qtd_operacoes = df_acao.shape[0]
taxa_acerto = df_acao['resultado_binario'].sum()/qtd_operacoes
operacoes_certas = qtd_operacoes * taxa_acerto
operacoes_erradas = qtd_operacoes - operacoes_certas
pontos_bruto =  (operacoes_certas * rate_tp - operacoes_erradas * rate_stop) * pontos

pontos_liquido = pontos_bruto - spread * qtd_operacoes

print(f'Timeframe: {timeframe} --pontos: {pontos} --- rate_tp: {rate_tp} -- var_bb: {var_bb}')

print(f'Pontos_liquido: {pontos_liquido} -- Total_operacoes: {qtd_operacoes}')

df_acao['Hora'] = df_acao['Data'].apply(lambda x: x[11:])
df_acao['Hora_h'] = df_acao['Hora'].apply(lambda x: x[:2])
df_acao['Mes'] = df_acao['Data'].apply(lambda x: x[5:7])

df_new_mes = df_acao[['Mes','resultado_binario']]
df_new_hora = df_acao[['Hora_h','resultado_binario']]

def calc_pontos(x):
    qtd_operacoes = x['qtd_operacoes']
    taxa_acerto = x['taxa_acerto']
    operacoes_certas = qtd_operacoes * taxa_acerto
    operacoes_erradas = qtd_operacoes - operacoes_certas
    pontos_bruto =  (operacoes_certas * rate_tp - operacoes_erradas * rate_stop) * pontos

    pontos_liquido = pontos_bruto - spread * qtd_operacoes

    return pontos_liquido

print('Agrupado por Hora')
df_new_hora = df_new_hora.groupby(['Hora_h'],as_index=False).agg(qtd_operacoes=('resultado_binario', 'count'), taxa_acerto=('resultado_binario', 'mean'))

df_new_hora['pontos_liquido'] = df_new_hora.apply(calc_pontos, axis = 1)
print(df_new_hora)

print('Agrupado por Mes')
df_new_mes = df_new_mes.groupby(['Mes'],as_index=False).agg(qtd_operacoes=('resultado_binario', 'count'), taxa_acerto=('resultado_binario', 'mean'))

df_new_mes['pontos_liquido'] = df_new_mes.apply(calc_pontos, axis = 1)
print(df_new_mes)
print('')


Timeframe: 5 --pontos: 60 --- rate_tp: 3 -- var_bb: 1.8
Pontos_liquido: 1435.0 -- Total_operacoes: 85
Agrupado por Hora
  Hora_h  qtd_operacoes  taxa_acerto  pontos_liquido
0     09             11     0.272727             5.0
1     10             12     0.333333           180.0
2     11              9     0.111111          -345.0
3     12              6     0.666667           570.0
4     13              8     0.375000           200.0
5     14             10     0.600000           790.0
6     15              9     0.444444           375.0
7     16              9     0.111111          -345.0
8     17              9     0.222222          -105.0
9     18              2     0.500000           110.0
Agrupado por Mes
  Mes  qtd_operacoes  taxa_acerto  pontos_liquido
0  12             85     0.341176          1435.0



In [8]:
df_acao.tail(10)

,Data,Open,Close,acao,stop,tp,resultado_binario,Hora,Hora_h,Mes
99817,2022-12-21 12:45:00,109005.0,109125.0,sell,109165.0,109045.0,1.0,12:45:00,12,12
99831,2022-12-21 13:55:00,108935.0,108615.0,call,108575.0,108695.0,1.0,13:55:00,13,12
99851,2022-12-21 15:35:00,109020.0,109165.0,sell,109205.0,109085.0,0.0,15:35:00,15,12
99869,2022-12-21 17:05:00,109190.0,109045.0,call,109005.0,109125.0,0.0,17:05:00,17,12
99885,2022-12-22 09:00:00,109000.0,109475.0,sell,109515.0,109395.0,0.0,09:00:00,09,12
99896,2022-12-22 09:55:00,109380.0,109550.0,sell,109590.0,109470.0,0.0,09:55:00,09,12
99910,2022-12-22 11:05:00,109190.0,109005.0,call,108965.0,109085.0,0.0,11:05:00,11,12
99936,2022-12-22 13:15:00,109430.0,109530.0,sell,109570.0,109450.0,1.0,13:15:00,13,12
99952,2022-12-22 14:35:00,109290.0,109125.0,call,109085.0,109205.0,0.0,14:35:00,14,12
99984,2022-12-22 17:15:00,108900.0,109115.0,sell,109155.0,109035.0,0.0,17:15:00,17,12


In [ ]:
qtd_operacoes = df_acao.shape[0]
taxa_acerto = df_acao['resultado_binario'].sum()/qtd_operacoes
operacoes_certas = qtd_operacoes * taxa_acerto
operacoes_erradas = qtd_operacoes - operacoes_certas
pontos_bruto =  (operacoes_certas * rate_tp - operacoes_erradas * rate_stop) * pontos

pontos_liquido = pontos_bruto - spread * qtd_operacoes


df_new = pd.DataFrame([[ativo[:-5],var_bb,rate_tp,rate_stop,pontos,qtd_operacoes,taxa_acerto,pontos_bruto,pontos_liquido]],
            columns = ['ativo','var_bb','rate_tp','rate_stop','pontos','qtd_operacoes','taxa_acerto','pontos_bruto','pontos_liquido'])

df_total = pd.concat([df_total,df_new])

print(df_total.to_string())